In [1]:
# Cell 1: Initialize Enhanced EnphaseClient
import sys
sys.path.append('../src')

from enphase_client import EnphaseClient
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path

print("=== ENPHASE CLIENT COMPREHENSIVE TEST SUITE ===\n")

# Load credentials
env_file = Path("../.env")
credentials = {}
with open(env_file, 'r') as f:
    for line in f:
        if '=' in line:
            key, value = line.strip().split('=', 1)
            credentials[key] = value

# Initialize client
client = EnphaseClient(
    access_token=credentials['ENPHASE_ACCESS_TOKEN'],
    api_key=credentials['ENPHASE_API_KEY'],
    system_id=credentials['ENPHASE_SYSTEM_ID']
)

print("EnphaseClient initialized with full API capabilities")

=== ENPHASE CLIENT COMPREHENSIVE TEST SUITE ===

EnphaseClient initialized with full API capabilities


/Users/christopherhaught/projects/power-usage-and-consumption/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Cell 2: Basic API functionality and health checks
print("=== API HEALTH AND SYSTEM STATUS ===\n")

# API health check
health = client.check_api_health()
if health['api_accessible']:
    print(f"API Status: Accessible")
    print(f"Current Power: {health['current_power']} W")
    print(f"Energy Today: {health['energy_today']:.1f} kWh")
    print(f"Lifetime Energy: {health['energy_lifetime']:.1f} MWh")
    print(f"Last Report: {health['last_report']}")
else:
    print(f"API Issue: {health.get('error')}")

# Detailed system status
print(f"\n--- Detailed System Information ---")
status = client.get_current_status()
if status:
    print(f"System ID: {status.get('system_id')}")
    print(f"Status: {status.get('status', 'Unknown')}")
    print(f"Size: {status.get('size_w', 'Unknown')} W")
    print(f"Module Count: {status.get('modules', 'Unknown')}")

=== API HEALTH AND SYSTEM STATUS ===

API Status: Accessible
Current Power: 0 W
Energy Today: 53.1 kWh
Lifetime Energy: 33.5 MWh
Last Report: 1758252162

--- Detailed System Information ---
System ID: 4136754
Status: normal
Size: 9425 W
Module Count: 26


In [3]:
# Cell 3: Comprehensive historical data analysis
print("=== HISTORICAL DATA ACCESS ===\n")

# Test full system lifetime data
print("1. Complete System History:")
full_data = client.get_historical_data()

if not full_data.empty:
    print(f"   Complete dataset: {len(full_data)} days")
    print(f"   Date range: {full_data.index.min().strftime('%Y-%m-%d')} to {full_data.index.max().strftime('%Y-%m-%d')}")
    print(f"   Total lifetime: {full_data['daily_energy_kwh'].sum():.1f} kWh")
    print(f"   Average daily: {full_data['daily_energy_kwh'].mean():.1f} kWh")
    print(f"   Peak day: {full_data['daily_energy_kwh'].max():.1f} kWh")

# Test recent data window
print(f"\n2. Recent 30-Day Analysis:")
recent_data = client.get_energy_lifetime(
    start_date=datetime.now() - timedelta(days=30),
    end_date=datetime.now()
)

if not recent_data.empty:
    print(f"   Recent period: {len(recent_data)} days")
    print(f"   Total energy: {recent_data['daily_energy_kwh'].sum():.1f} kWh")
    print(f"   Average daily: {recent_data['daily_energy_kwh'].mean():.1f} kWh")

    print(f"\n   Last 7 days production:")
    for date, row in recent_data.tail(7).iterrows():
        print(f"     {date.strftime('%Y-%m-%d')}: {row['daily_energy_kwh']:.1f} kWh")

=== HISTORICAL DATA ACCESS ===

1. Complete System History:
   Complete dataset: 889 days
   Date range: 2023-04-13 to 2025-09-17
   Total lifetime: 33499.2 kWh
   Average daily: 37.7 kWh
   Peak day: 61.7 kWh

2. Recent 30-Day Analysis:
   Recent period: 30 days
   Total energy: 1183.1 kWh
   Average daily: 39.4 kWh

   Last 7 days production:
     2025-09-11: 21.9 kWh
     2025-09-12: 50.9 kWh
     2025-09-13: 50.7 kWh
     2025-09-14: 26.6 kWh
     2025-09-15: 23.6 kWh
     2025-09-16: 16.7 kWh
     2025-09-17: 33.8 kWh


In [4]:
# Cell 4: Seasonal analysis and performance patterns
print("=== SEASONAL ANALYSIS ===\n")

if 'full_data' in locals() and not full_data.empty:
    # Monthly performance analysis
    full_data['month'] = full_data.index.month
    monthly_stats = full_data.groupby('month')['daily_energy_kwh'].agg(['mean', 'max', 'min', 'count'])

    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    print("Monthly Production Statistics:")
    for month_num, stats in monthly_stats.iterrows():
        month_name = months[month_num-1]
        print(f"  {month_name}: Avg={stats['mean']:.1f} kWh/day, Peak={stats['max']:.1f} kWh, Days={int(stats['count'])}")

    # Seasonal insights
    peak_month = monthly_stats['mean'].idxmax()
    low_month = monthly_stats['mean'].idxmin()

    print(f"\nSeasonal Performance:")
    print(f"  Peak production month: {months[peak_month-1]} ({monthly_stats.loc[peak_month, 'mean']:.1f} kWh/day)")
    print(f"  Lowest production month: {months[low_month-1]} ({monthly_stats.loc[low_month, 'mean']:.1f} kWh/day)")
    print(f"  Seasonal variation: {(monthly_stats['mean'].max() / monthly_stats['mean'].min() - 1) * 100:.0f}% difference")

    # Year-over-year comparison if multiple years available
    if len(full_data) > 365:
        full_data['year'] = full_data.index.year
        yearly_totals = full_data.groupby('year')['daily_energy_kwh'].sum()

        print(f"\nYear-over-Year Production:")
        for year, total in yearly_totals.items():
            print(f"  {year}: {total:.0f} kWh")

=== SEASONAL ANALYSIS ===

Monthly Production Statistics:
  Jan: Avg=30.4 kWh/day, Peak=45.0 kWh, Days=62
  Feb: Avg=32.8 kWh/day, Peak=50.9 kWh, Days=57
  Mar: Avg=38.8 kWh/day, Peak=58.3 kWh, Days=62
  Apr: Avg=33.6 kWh/day, Peak=59.9 kWh, Days=78
  May: Avg=40.4 kWh/day, Peak=61.7 kWh, Days=93
  Jun: Avg=45.9 kWh/day, Peak=59.2 kWh, Days=90
  Jul: Avg=42.9 kWh/day, Peak=58.2 kWh, Days=93
  Aug: Avg=40.8 kWh/day, Peak=57.5 kWh, Days=93
  Sep: Avg=37.8 kWh/day, Peak=57.9 kWh, Days=77
  Oct: Avg=40.8 kWh/day, Peak=53.4 kWh, Days=62
  Nov: Avg=31.7 kWh/day, Peak=48.9 kWh, Days=60
  Dec: Avg=27.6 kWh/day, Peak=41.2 kWh, Days=62

Seasonal Performance:
  Peak production month: Jun (45.9 kWh/day)
  Lowest production month: Dec (27.6 kWh/day)
  Seasonal variation: 66% difference

Year-over-Year Production:
  2023: 9704 kWh
  2024: 13675 kWh
  2025: 10120 kWh


In [5]:
# Cell 5: Revenue-grade meter detailed analysis
print("=== REVENUE-GRADE METER ANALYSIS ===\n")

# Test RGM stats functionality
print("1. Testing Multi-Channel Meter Data:")
rgm_data = client.get_recent_detailed_data(days_back=2)

if not rgm_data.empty:
    print(f"   Retrieved {len(rgm_data)} 15-minute intervals")
    print(f"   Date range: {rgm_data.index.min()} to {rgm_data.index.max()}")

    # Channel analysis
    channels = rgm_data['channel'].unique()
    print(f"   Meter channels detected: {sorted(channels)}")

    # Per-channel totals
    channel_totals = rgm_data.groupby('channel')['energy_delivered_kwh'].sum()
    print(f"\n   Energy by channel (last 2 days):")
    for channel in sorted(channels):
        total = channel_totals.get(channel, 0)
        print(f"     Channel {channel}: {total:.1f} kWh")

    # Hardware information
    if not rgm_data.empty:
        meter_info = rgm_data[['meter_serial', 'envoy_serial']].iloc[0]
        print(f"\n   Hardware Information:")
        print(f"     Meter Serial: {meter_info['meter_serial']}")
        print(f"     Envoy Serial: {meter_info['envoy_serial']}")

# Test power monitoring capabilities
print(f"\n2. Real-Time Power Monitoring:")
if not rgm_data.empty:
    # Filter for recent production hours
    recent_production = rgm_data[rgm_data['current_power_w'] > 0]

    if not recent_production.empty:
        peak_power = recent_production['current_power_w'].max()
        avg_power = recent_production['current_power_w'].mean()

        print(f"   Peak power reading: {peak_power:.0f} W")
        print(f"   Average power (when producing): {avg_power:.0f} W")

        # Show power progression during peak hours
        peak_day_data = recent_production.groupby(recent_production.index.date).first()
        if not peak_day_data.empty:
            print(f"   Latest production day sample:")
            sample_data = recent_production.tail(6)  # Last 6 intervals
            for dt, row in sample_data.iterrows():
                print(f"     {dt.strftime('%H:%M')} Ch{row['channel']}: {row['current_power_w']:.0f}W")

=== REVENUE-GRADE METER ANALYSIS ===

1. Testing Multi-Channel Meter Data:
   Retrieved 576 15-minute intervals
   Date range: 2025-09-17 03:30:00 to 2025-09-19 03:15:00
   Meter channels detected: [np.int64(1), np.int64(2), np.int64(3)]

   Energy by channel (last 2 days):
     Channel 1: 43.4 kWh
     Channel 2: 43.5 kWh
     Channel 3: 0.0 kWh

   Hardware Information:
     Meter Serial: 202251123433EIM1
     Envoy Serial: 202251123433

2. Real-Time Power Monitoring:
   Peak power reading: 3833 W
   Average power (when producing): 1838 W
   Latest production day sample:
     22:30 Ch2: 125W
     22:30 Ch1: 126W
     22:45 Ch2: 90W
     22:45 Ch1: 91W
     23:00 Ch2: 34W
     23:00 Ch1: 35W
